# NYC Taxi & Limousine Commission (TLC) Data Analysis

This notebook provides a comprehensive analysis of NYC TLC trip data, exploring various aspects of taxi operations including:
- Trip Volume Patterns
- Financial Metrics
- Geographic Distribution
- Performance Indicators

## Setup and Dependencies

In [ ]:
import pandas as pd
import numpy as np
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots
from google.cloud import bigquery
from datetime import datetime, timedelta

# Initialize BigQuery client
client = bigquery.Client(project='scenic-flux-441021-t4')

## 1. Trip Volume Analysis

Let's analyze the patterns in trip volume across different time periods.

In [ ]:
# Query trip volume data
query = """
SELECT * FROM `scenic-flux-441021-t4.nyc_tlc_data.dashboard_trip_volume`
"""
volume_df = client.query(query).to_dataframe()

# Hourly patterns
fig = px.line(volume_df, x='pickup_hour', y='trip_count',
              title='Hourly Trip Distribution',
              labels={'pickup_hour': 'Hour of Day',
                     'trip_count': 'Number of Trips'})
fig.show()

# Daily patterns with average fare
fig = make_subplots(specs=[[{"secondary_y": True}]])
fig.add_trace(
    go.Bar(x=volume_df['pickup_dayofweek'], y=volume_df['trip_count'],
           name="Trip Count"),
    secondary_y=False
)
fig.add_trace(
    go.Scatter(x=volume_df['pickup_dayofweek'], y=volume_df['avg_fare'],
               name="Average Fare", mode='lines+markers'),
    secondary_y=True
)
fig.update_layout(title='Daily Trip Patterns and Average Fares')
fig.show()

### Key Observations:
- Peak hours typically occur during rush hours (morning and evening)
- Weekend patterns differ from weekdays
- Average fares tend to be higher during off-peak hours

## 2. Financial Analysis

Let's examine the financial aspects of the taxi operations.

In [ ]:
# Query financial metrics
query = """
SELECT * FROM `scenic-flux-441021-t4.nyc_tlc_data.dashboard_financial_metrics`
"""
finance_df = client.query(query).to_dataframe()

# Payment type distribution
fig = px.pie(finance_df, values='trip_count', names='payment_type',
             title='Trip Distribution by Payment Type')
fig.show()

# Tip percentage patterns
fig = px.violin(finance_df, x='time_of_day', y='avg_tip_percentage',
                title='Tip Percentage Distribution by Time of Day',
                labels={'time_of_day': 'Time of Day',
                       'avg_tip_percentage': 'Average Tip (%)'})
fig.show()

### Financial Insights:
- Credit card payments dominate the payment methods
- Tip percentages vary significantly by time of day
- Higher tips are generally observed during evening hours

## 3. Geographic Distribution

Analyzing the spatial patterns of taxi trips.

In [ ]:
# Query geographic metrics
query = """
SELECT * FROM `scenic-flux-441021-t4.nyc_tlc_data.dashboard_geographic_metrics`
"""
geo_df = client.query(query).to_dataframe()

# Top pickup locations
top_locations = geo_df.groupby('pickup_location_id').agg({
    'trip_count': 'sum',
    'avg_fare': 'mean'
}).sort_values('trip_count', ascending=False).head(10)

fig = px.bar(top_locations, x=top_locations.index, y='trip_count',
             title='Top 10 Pickup Locations',
             labels={'index': 'Location ID',
                    'trip_count': 'Number of Trips'})
fig.show()

# Speed vs Distance patterns
fig = px.scatter(geo_df, x='avg_distance', y='avg_speed',
                 color='avg_fare',
                 title='Speed vs Distance by Location',
                 labels={'avg_distance': 'Average Distance (miles)',
                        'avg_speed': 'Average Speed (mph)',
                        'avg_fare': 'Average Fare ($)'})
fig.show()

### Geographic Insights:
- Certain locations consistently generate more trips
- Trip speed varies with distance and location
- Higher fares correlate with longer distances

## 4. Performance Metrics

Analyzing operational performance indicators.

In [ ]:
# Query performance metrics
query = """
SELECT * FROM `scenic-flux-441021-t4.nyc_tlc_data.dashboard_performance_metrics`
"""
perf_df = client.query(query).to_dataframe()

# Speed distribution by time
fig = px.box(perf_df, x='time_of_day', y='avg_speed',
             color='is_weekend',
             title='Speed Distribution by Time',
             labels={'time_of_day': 'Time of Day',
                    'avg_speed': 'Average Speed (mph)',
                    'is_weekend': 'Weekend'})
fig.show()

# Trip duration patterns
fig = px.violin(perf_df, x='trip_distance_category', y='avg_duration',
                color='is_weekend',
                title='Trip Duration by Distance Category',
                labels={'trip_distance_category': 'Distance Category',
                       'avg_duration': 'Average Duration (minutes)',
                       'is_weekend': 'Weekend'})
fig.show()

### Performance Insights:
- Average speeds vary significantly by time of day
- Weekend trips show different duration patterns
- Trip duration correlates strongly with distance category

## Summary

This analysis reveals several key insights about NYC taxi operations:

1. **Temporal Patterns**
   - Clear rush hour peaks in trip volume
   - Different patterns for weekdays vs weekends

2. **Financial Trends**
   - Credit card payments are most common
   - Tips vary by time and location

3. **Geographic Distribution**
   - Certain locations generate significantly more trips
   - Trip characteristics vary by location

4. **Operational Performance**
   - Speed and duration affected by time of day
   - Weekend operations show distinct patterns

These insights can be valuable for:
- Optimizing taxi fleet distribution
- Improving service during peak hours
- Understanding customer behavior
- Enhancing operational efficiency